**Part 1: Retrieval-Augmented Generation (RAG) Model for QA Bot and P&L Data**

**Installations:**

In [4]:
!pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib
!sudo apt-get install poppler-utils libleptonica-dev tesseract-ocr libtesseract-dev
!pip install unstructured-pytesseract tesseract-ocr
!pip install --upgrade nltk
!pip install datasets transformers langchain langchain_community sentence-transformers accelerate pinecone-client google-generativea

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached langchain_community-0.3.15-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement google-generativea (from versions: none)
ERROR: No matching distribution found for google-generativea


In [5]:
!pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib
!sudo apt-get install poppler-utils libleptonica-dev tesseract-ocr libtesseract-dev
!pip install unstructured-pytesseract tesseract-ocr
!pip install --upgrade nltk
!pip install datasets transformers langchain langchain_community sentence-transformers accelerate google-generativea
!pip install pinecone-client # Install pinecone-client separately to ensure it is installed


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached langchain_community-0.3.15-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement google-generativea (from versions: none)
ERROR: No matching distribution found for google-generativea


**Imports:**

In [6]:
import os
import warnings
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import pinecone
import google.generativeai as genai
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Table
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nltk
import string

warnings.filterwarnings('ignore')
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [7]:
# Pinecone and Google Gemini API keys
PINECONE_API_KEY = "YOUR PINECONE API KEY"
GOOGLE_API_KEY = "YOUR GOOGLE API KEY"

# Set Google API key
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

# Configure Google Gemini API
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [8]:
# Load embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

**PDF Table Extraction:**

In [9]:
# Install the required packages
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [10]:
!apt-get update
!apt-get install -y poppler-utils


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [11]:
!sudo apt-get install -y poppler-utils libleptonica-dev tesseract-ocr libtesseract-dev
!pip install unstructured-pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [12]:
# Extract various elements from pdf
from unstructured.partition.pdf import partition_pdf

raw_pdf_elements = partition_pdf(
    filename="/content/Sample Financial Statement.pdf",
    strategy="hi_res",
    hi_res_model_name="yolox",
    skip_infer_table_types=[],
    pdf_infer_table_structure=True,
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data",
    ocr_languages="eng"  # Set OCR language to English
)


In [13]:
# Display raw extracted elements
for element in raw_pdf_elements:
    print(element)



INFOSYS LIMITED AND SUBSIDIARIES Condensed Consolidated Financial Statements under Indian Accounting Standards (Ind AS) for the three months and year ended March 31, 2024
Index
Page No.
1 2 3 5 7 7 7 7 8 2.21 Contingent liabilities and commitments ……………………………………………………………………………… 34
Condensed Consolidated Balance Sheet ……………………………………………………………………………………………………………………..
Condensed Consolidated Statement of Profit and Loss ……………………………………………………………………………………………………………………..
Condensed Consolidated Statement of Changes in Equity ……………………………………………………………………………………………………………………..
Condensed Consolidated Statement of Cash Flows ……………………………………………………………………………………………………………………..
Overview and Notes to the Interim Condensed Consolidated Financial Statements
1. Overview
1.1 Company overview ……………………………………………………………………………………………………………………..
1.2 Basis of preparation of financial statements ……………………………………………………………………………………………………………………..
1.3 Basis of consolidation ……………………………………………………………………………………………………………………..
1.4 Use of e

In [14]:
for idx, element in enumerate(raw_pdf_elements):
    print(f"Element {idx}: {element}")
    print(f"Type: {type(element)}")
    print(f"Text: {getattr(element, 'text', 'No text')}")


The amount received in excess of the par value of equity shares has been classified as securities premium. Amounts have been utilized for bonus issue and share buyback from share premium account.
Share options outstanding account
The share options outstanding account is used to record the fair value of equity-settled share based payment transactions with employees. The amounts recorded in share options outstanding account are transferred to securities premium upon exercise of stock options and transferred to general reserve on account of stock options not exercised by employees.
Special Economic Zone Re-investment reserve
The Special Economic Zone Re-investment reserve has been created out of the profit of the eligible SEZ unit in terms of the provisions of Sec 10AA (1)(ii) of Income Tax Act, 1961. The reserve should be utilized by the Company for acquiring new plant and machinery for the purpose of its business in terms of the provisions of the Sec 10AA (2) of the Income Tax Act, 1961

In [15]:
print(f"Total elements extracted: {len(raw_pdf_elements)}")
print(raw_pdf_elements[:5])  # Preview first few elements


Total elements extracted: 1014
[<unstructured.documents.elements.Title object at 0x7e20fbf0e790>, <unstructured.documents.elements.Title object at 0x7e20fb873cd0>, <unstructured.documents.elements.Title object at 0x7e20fb873f90>, <unstructured.documents.elements.Table object at 0x7e20fb873590>, <unstructured.documents.elements.Text object at 0x7e21002a4c90>]


In [16]:
# Count the unique types of elements in raw_pdf_elements
from collections import Counter

# Extract the type of each element
element_types = [type(element).__name__ for element in raw_pdf_elements]

# Count occurrences of each type
type_counts = Counter(element_types)

# Print the counts
print("Unique Element Types and Their Counts:")
for element_type, count in type_counts.items():
    print(f"{element_type}: {count}")


Unique Element Types and Their Counts:
Title: 246
Table: 50
Text: 325
ListItem: 31
NarrativeText: 324
Footer: 27
Header: 10
FigureCaption: 1


In [17]:
# Extract tables from pdf
def extract_tables(elements):
    """
    Extract tables using HTML parsing with fallback to text-based methods.
    """
    tables = [el for el in elements if el.category == "Table"]
    extracted_tables = []

    for table in tables:
        try:
            # Attempt to parse HTML directly
            table_html = table.metadata.get("text_as_html", "")
            if table_html:
                dfs = pd.read_html(table_html)
                extracted_tables.extend(dfs)  # Add all parsed DataFrames
            else:
                raise ValueError("No valid HTML found for table.")
        except Exception as e:
            # Fallback to text-based parsing
            try:
                table_text = table.text
                rows = table_text.split("\n")
                table_data = [row.split() for row in rows if row.strip()]
                df = pd.DataFrame(table_data)
                extracted_tables.append(df)
            except Exception as fallback_error:
                print(f"Error extracting table: {fallback_error}")
                continue

    return extracted_tables


**Cleaning Tables:**

In [18]:
# Clean the extracted tables
def clean_table(table):


    if table is not None:
        # Drop empty rows/columns
        table.dropna(how="all", axis=0, inplace=True)
        table.dropna(how="all", axis=1, inplace=True)

        # Reset index for consistency
        table.reset_index(drop=True, inplace=True)

    return table


**Convert a table to dictionary:**

In [19]:
def convert_table_to_dict(table):
    """
    Convert a DataFrame into a list of dictionaries representing rows.
    """
    if table.empty:
        return []

    # Ensure headers are correctly set
    headers = table.columns
    structured_data = [
        {str(header): str(value) if pd.notnull(value) else "" for header, value in zip(headers, row)}
        for _, row in table.iterrows()
    ]
    return structured_data

**Summarize a table using Gemini:**

In [20]:
import signal

# Define a timeout handler
def handler(signum, frame):
    raise TimeoutError("Gemini summarization took too long.")

# Summarize function with timeout
def summarize_table_with_gemini(table_html):
    """
    Summarize table content using Gemini.
    """
    prompt = f"""
    You are a financial assistant analyzing financial data. Extract and summarize key information from the following table:

    {table_html}

    Include details such as totals, liabilities, revenues, profits, and any financial trends or anomalies.
    """
    try:
        response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Error in Gemini summarization: {e}")
        return "Summary unavailable."


**Embedding and Metadata Generation:**

In [21]:
def process_tables_with_gemini_and_embeddings(cleaned_tables):
    """
    Process cleaned tables, generate embeddings, and summarize using Gemini.
    """
    summaries = []
    embeddings = []

    for idx, table in enumerate(cleaned_tables, start=1):
        try:
            # Convert table to HTML
            table_html = table.to_html(index=False)
            summary = summarize_table_with_gemini(table_html)
            print(f"Table {idx} summarized.")

            # Generate embeddings for table and summary
            summary_embedding = embedding_model.encode(summary)
            table_embedding = embedding_model.encode(table_html)

            summaries.append({
                "summary": summary,
                "embedding": summary_embedding,
                "table": table,
                "table_embedding": table_embedding
            })
        except Exception as e:
            print(f"Error processing table {idx}: {e}")

    return summaries


**Pinecone Setup**

In [22]:
import os
from pinecone import Pinecone, ServerlessSpec

# Upsert Combined Embeddings into Pinecone
import pinecone



def upsert_to_pinecone(summaries, index):
    """
    Robust upsert method with enhanced error handling and type conversion.
    """
    vectors = []

    for idx, summary in enumerate(summaries):
        try:
            # Standardize table data conversion
            table_metadata = _convert_table_to_pinecone_metadata(summary["table"])

            # Add summary embedding vector
            vectors.append({
                "id": f"summary_{idx}",
                "values": summary["embedding"].tolist(),
                "metadata": {
                    "summary": summary["summary"] or "No summary available",
                    "type": "summary",
                    "table_data": table_metadata
                }
            })

        except Exception as e:
            print(f"Error preparing vector for table {idx}: {e}")
            continue

    if vectors:
        try:
            index.upsert(vectors=vectors)
            print(f"Successfully upserted {len(vectors)} vectors.")
        except Exception as e:
            print(f"Pinecone upsert error: {e}")
    else:
        print("No vectors to upsert.")

def _convert_table_to_pinecone_metadata(table):
    """
    Robust conversion of table to Pinecone-compatible metadata.
    """
    if table is None or table.empty:
        return "No table data available"

    try:
        # Convert DataFrame to list of string representations
        table_metadata = []
        for _, row in table.iterrows():
            row_dict = row.dropna().to_dict()
            row_str = " | ".join([f"{k}: {v}" for k, v in row_dict.items()])
            table_metadata.append(row_str)

        return " || ".join(table_metadata)
    except Exception as e:
        print(f"Table conversion error: {e}")
        return "Table conversion failed"











**Get the relevant context based on user input:**

In [23]:
# RAG Retrieval and Reranking
def get_relevant_contexts(query, index):
    """
    Retrieve relevant rows or summaries from Pinecone with improved error handling.
    """
    query_embedding = embedding_model.encode(query).tolist()
    try:
        results = index.query(vector=query_embedding, top_k=10, include_metadata=True)
        contexts = []

        for match in results.get("matches", []):
            summary = match["metadata"].get("summary", "No summary available")
            table_data = match["metadata"].get("table_data", "No table data")

            contexts.append((summary, table_data))

        return contexts
    except Exception as e:
        print(f"Error querying index: {e}")
        return []










**Generate the answer using Gemini AI model:**

In [24]:
def generate_answer_with_gemini(contexts, question):
    """
    Enhanced answer generation with robust context processing.
    """
    summaries = []
    table_contexts = []

    for ctx in contexts:
        summary, table_data = ctx
        if summary:
            summaries.append(summary)

        if table_data and isinstance(table_data, str):
            table_contexts.append(table_data)

    summary_context = "\n".join(summaries)
    table_context = "\n".join(table_contexts)

    prompt = f"""
    Financial Analysis Context:
    Summaries: {summary_context}
    Table Data: {table_context}

    Question: {question}
    Provide a precise, data-driven answer.
    """

    try:
        response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Answer generation error: {e}")
        return "Unable to generate a comprehensive answer based on available data."



**Main function Execution:**

In [25]:
def main():
    print("Welcome to the Financial QA Bot!")
    print("Type 'exit' to quit.")

    # Step 1: Extract tables from the PDF
    print("\nExtracting tables from the PDF...")
    extracted_tables = extract_tables(raw_pdf_elements)
    print(f"Extracted {len(extracted_tables)} tables.")

    # Step 2: Clean the extracted tables
    print("\nCleaning extracted tables...")
    cleaned_tables = [clean_table(table) for table in extracted_tables]
    print(f"Cleaned {len(cleaned_tables)} tables.")

    # Step 3: Summarize and embed tables
    print("\nSummarizing and embedding tables...")
    processed_data = process_tables_with_gemini_and_embeddings(cleaned_tables)
    print(f"Processed {len(processed_data)} tables.")

    # Step 4: Initialize Pinecone and upsert data
    print("\nInitializing Pinecone...")
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "financial-tables"
    if index_name not in [idx.name for idx in pc.list_indexes()]:
        pc.create_index(
            name=index_name,
            dimension=len(processed_data[0]["embedding"]),
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-west-2")
        )
    index = pc.Index(index_name)
    print(f"Index '{index_name}' retrieved successfully.")

    print("\nUpserting data to Pinecone...")
    upsert_to_pinecone(processed_data, index)
    print("Data successfully upserted to Pinecone.")

    # Step 5: Enter chatbot loop
    print("\nChatbot is ready. Ask your financial questions!")
    while True:
        user_input = input("\nUser: ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        # Retrieve relevant contexts
        print("\nRetrieving relevant contexts...")
        contexts = get_relevant_contexts(user_input, index)
        if not contexts:
            print("No relevant context found. Please refine your question.")
            continue

        # Debug retrieved contexts
        print("\n--- Debugging Retrieved Contexts ---")
        for ctx in contexts:
            print(f"Summary: {ctx[0][:100]}")
            print(f"Table Data: {ctx[1]}")

        # Generate an answer using Gemini
        print("\nGenerating answer...")
        answer = generate_answer_with_gemini(contexts, user_input)
        print(f"AI response: {answer}")


if __name__ == "__main__":
    main()


Welcome to the Financial QA Bot!
Type 'exit' to quit.

Extracting tables from the PDF...
Extracted 50 tables.

Cleaning extracted tables...
Cleaned 50 tables.

Summarizing and embedding tables...
Table 1 summarized.
Table 2 summarized.
Table 3 summarized.
Table 4 summarized.
Table 5 summarized.
Table 6 summarized.
Table 7 summarized.
Table 8 summarized.
Table 9 summarized.
Table 10 summarized.
Table 11 summarized.
Table 12 summarized.
Table 13 summarized.
Table 14 summarized.
Table 15 summarized.
Table 16 summarized.
Table 17 summarized.
Table 18 summarized.
Table 19 summarized.
Table 20 summarized.
Table 21 summarized.
Table 22 summarized.
Table 23 summarized.
Table 24 summarized.
Table 25 summarized.
Table 26 summarized.
Table 27 summarized.
Table 28 summarized.
Table 29 summarized.
Table 30 summarized.
Table 31 summarized.
Table 32 summarized.
Table 33 summarized.
Table 34 summarized.
Table 35 summarized.
Table 36 summarized.
Table 37 summarized.
Table 38 summarized.
Table 39 summar

**Part 2: Interactive QA Bot Interface for Financial Data**

**Installations:**

In [ ]:
!pip install streamlit pyngrok


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 4s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
!wget -q -O - ipv4.icanhazip.com  # This will give us the password for local tunnel

34.86.25.175


**Streamlit application set up:**

In [ ]:
%%writefile app.py
import streamlit as st
import tempfile
import os
from unstructured.partition.pdf import partition_pdf
import signal
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import pinecone
import google.generativeai as genai
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Table
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nltk
import string
import warnings

warnings.filterwarnings('ignore')
nltk.download('all')




# Pinecone and Google Gemini API keys
PINECONE_API_KEY = "YOUR PINECONE API KEY"
GOOGLE_API_KEY = "YOUR GOOGLE API KEY"

# Set Google API key
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

# Configure Google Gemini API
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])


# Load embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

#Initialize Pinecone
import os
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "financial-tables"
if index_name not in [idx.name for idx in pc.list_indexes()]:
         pc.create_index(
            name=index_name,
            dimension=768,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-west-2")
            )
index = pc.Index(index_name)


# Initialize Streamlit session state
if "raw_pdf_elements" not in st.session_state:
    st.session_state.raw_pdf_elements = None
if "cleaned_tables" not in st.session_state:
    st.session_state.cleaned_tables = None
if "processed_data" not in st.session_state:
    st.session_state.processed_data = None
if "pinecone_index" not in st.session_state:
    st.session_state.pinecone_index = index


# Define the utility function
def extract_tables(elements):
    """
    Extract tables using HTML parsing with fallback to text-based methods.
    """
    tables = [el for el in elements if el.category == "Table"]
    extracted_tables = []

    for table in tables:
        try:
            # Attempt to parse HTML directly
            table_html = table.metadata.get("text_as_html", "")
            if table_html:
                dfs = pd.read_html(table_html)
                extracted_tables.extend(dfs)  # Add all parsed DataFrames
            else:
                raise ValueError("No valid HTML found for table.")
        except Exception as e:
            # Fallback to text-based parsing
            try:
                table_text = table.text
                rows = table_text.split("\n")
                table_data = [row.split() for row in rows if row.strip()]
                df = pd.DataFrame(table_data)
                extracted_tables.append(df)
            except Exception as fallback_error:
                print(f"Error extracting table: {fallback_error}")
                continue

    return extracted_tables



# Clean the extracted tables
def clean_table(table):
    """
    Minimize cleaning to retain table structure.
    """
    if table is not None:
        # Drop empty rows/columns
        table.dropna(how="all", axis=0, inplace=True)
        table.dropna(how="all", axis=1, inplace=True)

        # Reset index for consistency
        table.reset_index(drop=True, inplace=True)

    return table



# Summarize and embed the tables


# Define a timeout handler
def handler(signum, frame):
    raise TimeoutError("Gemini summarization took too long.")

# Summarize function with timeout
def summarize_table_with_gemini(table_html):
    """
    Summarize table content using Gemini.
    """
    prompt = f"""
    You are a financial assistant analyzing financial data. Extract and summarize key information from the following table:

    {table_html}

    Include details such as totals, liabilities, revenues, profits, and any financial trends or anomalies.
    """
    try:
        response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        st.write(f"Error in Gemini summarization: {e}")
        return "Summary unavailable."

def process_tables_with_gemini_and_embeddings(cleaned_tables):
    """
    Process cleaned tables, generate embeddings, and summarize using Gemini.
    """
    summaries = []
    embeddings = []

    for idx, table in enumerate(cleaned_tables, start=1):
        try:
            # Convert table to HTML
            table_html = table.to_html(index=False)
            summary = summarize_table_with_gemini(table_html)
            st.write(f"Table {idx} summarized.")

            # Generate embeddings for table and summary
            summary_embedding = embedding_model.encode(summary)
            table_embedding = embedding_model.encode(table_html)

            summaries.append({
                "summary": summary,
                "embedding": summary_embedding,
                "table": table,
                "table_embedding": table_embedding
            })
        except Exception as e:
            st.write(f"Error processing table {idx}: {e}")

    return summaries


def upsert_to_pinecone(summaries, index):
    """
    Robust upsert method with enhanced error handling and type conversion.
    """
    vectors = []

    for idx, summary in enumerate(summaries):
        try:
            # Standardize table data conversion
            table_metadata = _convert_table_to_pinecone_metadata(summary["table"])

            # Add summary embedding vector
            vectors.append({
                "id": f"summary_{idx}",
                "values": summary["embedding"].tolist(),
                "metadata": {
                    "summary": summary["summary"] or "No summary available",
                    "type": "summary",
                    "table_data": table_metadata
                }
            })

        except Exception as e:
            st.write(f"Error preparing vector for table {idx}: {e}")
            continue

    if vectors:
        try:
            index.upsert(vectors=vectors)
            st.write(f"Successfully upserted {len(vectors)} vectors.")
        except Exception as e:
            st.write(f"Pinecone upsert error: {e}")
    else:
        st.write("No vectors to upsert.")

def _convert_table_to_pinecone_metadata(table):
    """
    Robust conversion of table to Pinecone-compatible metadata.
    """
    if table is None or table.empty:
        return "No table data available"

    try:
        # Convert DataFrame to list of string representations
        table_metadata = []
        for _, row in table.iterrows():
            row_dict = row.dropna().to_dict()
            row_str = " | ".join([f"{k}: {v}" for k, v in row_dict.items()])
            table_metadata.append(row_str)

        return " || ".join(table_metadata)
    except Exception as e:
        st.write(f"Table conversion error: {e}")
        return "Table conversion failed"


# RAG Retrieval and Reranking
def get_relevant_contexts(query, index):
    """
    Retrieve relevant rows or summaries from Pinecone with improved error handling.
    """
    query_embedding = embedding_model.encode(query).tolist()
    try:
        results = index.query(vector=query_embedding, top_k=10, include_metadata=True)
        contexts = []

        for match in results.get("matches", []):
            summary = match["metadata"].get("summary", "No summary available")
            table_data = match["metadata"].get("table_data", "No table data")

            contexts.append((summary, table_data))

        return contexts
    except Exception as e:
        st.write(f"Error querying index: {e}")
        return []


def generate_answer_with_gemini(contexts, question):
    """
    Enhanced answer generation with robust context processing.
    """
    summaries = []
    table_contexts = []

    for ctx in contexts:
        summary, table_data = ctx
        if summary:
            summaries.append(summary)

        if table_data and isinstance(table_data, str):
            table_contexts.append(table_data)

    summary_context = "\n".join(summaries)
    table_context = "\n".join(table_contexts)

    prompt = f"""
    Financial Analysis Context:
    Summaries: {summary_context}
    Table Data: {table_context}

    Question: {question}
    Provide a precise, data-driven answer.
    """

    try:
        response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        st.write(f"Answer generation error: {e}")
        return "Unable to generate a comprehensive answer based on available data."


# Upsert Combined Embeddings into Pinecone
import pinecone







st.title("Financial QA Bot")
st.write("Upload a P&L statement in PDF format, ask financial questions, and get precise answers.")

# Step 1: Upload PDF File
uploaded_file = st.file_uploader("Upload your financial PDF document", type=["pdf"])
if uploaded_file:
    # Save the uploaded file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(uploaded_file.read())
        pdf_path = temp_file.name

    st.write("Processing the uploaded PDF...This will take a few minutes...Please be patient...")

    # Step 2: Extract tables with a spinner
    with st.spinner("Extracting tables from the PDF. Please wait..."):
        st.session_state.raw_pdf_elements = partition_pdf(
            filename=pdf_path,
            strategy="hi_res",
            hi_res_model_name="yolox",
            skip_infer_table_types=[],
            pdf_infer_table_structure=True,
            extract_images_in_pdf=True,
            extract_image_block_types=["Image", "Table"],
            extract_image_block_to_payload=False,
            extract_image_block_output_dir="extracted_data",
        )
    st.success("PDF processing completed successfully!")

    # Proceed to extract tables only after raw_pdf_elements is defined
    st.write(f"Extracting tables from {len(st.session_state.raw_pdf_elements)} elements...")
    extracted_tables = extract_tables(st.session_state.raw_pdf_elements)
    st.write(f"Extracted {len(extracted_tables)} tables.")

    # Clean the extracted tables
    st.write("Cleaning the extracted tables...")
    st.session_state.cleaned_tables = [clean_table(table) for table in extracted_tables]
    st.write(f"Cleaned {len(st.session_state.cleaned_tables)} tables.")
else:
    st.warning("Please upload a PDF file to start processing.")

# Summarize and embed tables only if not already processed
if "processed_data" not in st.session_state or st.session_state.processed_data is None:
    st.write("Summarizing and embedding the tables...")
    st.session_state.processed_data = process_tables_with_gemini_and_embeddings(st.session_state.cleaned_tables)
    st.write(f"Processed {len(st.session_state.processed_data)} tables.")
else:
    st.write("Tables have already been summarized and embedded.")

# Step 3: Initialize Pinecone
st.write("Initializing Pinecone...")
st.write(f"Index '{index_name}' retrieved successfully.")

# Upsert data to Pinecone only if not already done
if "pinecone_upsert_done" not in st.session_state:
    st.write("Upserting data to Pinecone...")
    upsert_to_pinecone(st.session_state.processed_data, st.session_state.pinecone_index)
    st.session_state.pinecone_upsert_done = True  # Mark as done
    st.write("Data successfully upserted to Pinecone.")
else:
    st.write("Data has already been upserted to Pinecone.")

# Initialize session state for questions and answers
if "qa_history" not in st.session_state:
    st.session_state.qa_history = []  # List to store questions and answers

st.write("The bot is ready. You can now ask your financial questions repeatedly. Type 'exit' to quit.")

# User interaction loop
user_query = st.text_input("Enter your question (type 'exit' to quit):")
if user_query:
    if user_query.lower() == "exit":
        st.write("Goodbye! Thank you for using the Financial QA Bot.")
    else:
        st.write("Retrieving relevant contexts...")
        contexts = get_relevant_contexts(user_query, st.session_state.pinecone_index)

        if not contexts:
            st.write("No relevant context found. Please refine your question.")
        else:
            # Filter the most relevant table or row
            most_relevant_table = contexts[0][1]  # Take the first match as most relevant
            st.write("Relevant Context:")
            st.write(most_relevant_table)  # Display only the relevant table or reference

            # Generate an answer
            st.write("Generating an answer...")
            answer = generate_answer_with_gemini(contexts, user_query)
            st.session_state.qa_history.append({"question": user_query, "answer": answer})
            st.write(f"AI Response: {answer}")

# Display full QA history for context
if st.session_state.qa_history:
    st.write("**Chat History:**")
    for i, qa in enumerate(st.session_state.qa_history, start=1):
        st.write(f"Q{i}: {qa['question']}")
        st.write(f"A{i}: {qa['answer']}")








Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

your url is: https://fifty-stars-thank.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.25.175:8501

2025-01-24 12:57:39.080003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 12:57:39.109534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 12:57:39.119430: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 12:57:40.550669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downlo